In [1]:
from graph_ds import PyH3Graph

In [2]:
graph = PyH3Graph()
graph.create(osm_path="../resources/copenhagen-processed.osm.pbf", gtfs_path="../resources/rejseplanen.zip")

processing osm pbf file: ../resources/copenhagen-processed.osm.pbf
converted OSM file into 446019 edges
osm graph created with 364511 nodes in 2 s
getting GTFS feed from ../resources/rejseplanen.zip
routes: 1631
gtfs graph created with 103525 nodes in 9 s
merged graph created with 465889 nodes in 0 s
hash: 12790899336573405047


In [3]:
graph.astar_path(631049690149925887, 631049690224658431)

nodes have been adjusted: 631049690224658431 -> 631049690224681983
astar from 631049690149925887 to 631049690224681983
[backtrace] found start node


([631049690149925887,
  631049690149927935,
  631049690149927423,
  631049690149913599,
  631049690149915647,
  631049690149915135,
  631049690149931007,
  631049690149940223,
  631049690149930495,
  631049690149884927,
  631049690149886975,
  631049690149823999,
  631049690149901311,
  631049690149903359,
  631049690149902847,
  631049690150024191,
  631049690150023679,
  631049690150025727,
  631049690150012927,
  631049690150011391,
  631049690150013439,
  631049690150029311,
  631049690150027775,
  631049690150030847,
  631049690150971391,
  631049690150970879,
  631049690150972927,
  631049690151574527,
  631049690150987263,
  631049690150989311,
  631049690150934527,
  631049690150934015,
  631049690150936063,
  631049690150931455,
  631049690150950399,
  631049690150945791,
  631049690150947839,
  631049690151073279,
  631049690151068671,
  631049690151070719,
  631049690151070207,
  631049690151085055,
  631049690151087103,
  631049690151077375,
  631049690162828287,
  63104969

In [8]:
origins = [631049690183383551, 631049690224672255, 631049690224671743, 631049690149913599, 631049690149925887]
destinations = [631049690149913599, 631049690149915135, 631049690149830143, 631049690183383551, 631049690224658431]
dist_result = graph.matrix_distance(origins, destinations, 9)

In [9]:
dist_result

{631049690224671743: [13.002393338999983,
  13.226572534499983,
  12.554034947999984,
  0.6725375865000001],
 631049690149913599: [0.2241791955,
  0.0,
  0.6725375865000001,
  13.899110120999982],
 631049690224672255: [13.226572534499983,
  13.450751729999983,
  12.778214143499984,
  0.448358391],
 631049690149925887: [0.56044798875,
  0.33626879325000003,
  1.0088063797500002,
  14.235378914249981]}

In [5]:
dist_result

{631049690224671743: [13.002393338999983,
  12.554034947999984,
  0.6725375865000001,
  13.226572534499983],
 631049690149913599: [0.2241791955,
  0.6725375865000001,
  13.899110120999982,
  0.0],
 631049690224672255: [13.226572534499983,
  12.778214143499984,
  0.448358391,
  13.450751729999983],
 631049690149925887: [0.56044798875,
  1.0088063797500002,
  14.235378914249981,
  0.33626879325000003]}